In [100]:
import pandas as pd
import ast

In [101]:
df_stops_1980 = pd.read_csv('../1980-network/stops_df_1980-final.csv')
df_1982 = pd.read_csv('stops_df_1982-initial.csv')

In [102]:
df_1982.head(3)

,Unnamed: 0,stop_name,line_count,type,in_lines
0,0,"Moabit, Waldstrasse Ecke Turmstrasse",1,bus,{'1'}
1,1,Turmstrasse,4,bus,"{'1', '70', '16'}"
2,2,Turmstrasse,4,u-bahn,{'9'}


In [103]:
df_1982.drop(columns=['Unnamed: 0'], inplace=True)
df_1982["location"] = ""
df_1982["identifier"] = ""
df_1982["stop_id_other"] = ""
df_1982["previous_in_lines"] = ""
df_1982["stop_description"] = ""

In [104]:
df_stops_1980.tail(3)

,Unnamed: 0,stop_id,stop_name,type,location,in_lines,identifier
1717,1717,19801717,Zwickauer Damm,u-bahn,"52.423333333333,13.483888888889",['7'],Q572096
1718,1718,19801718,Zwieselerstrasse,bus,"52.49612228648307, 13.540160888423404",['94'],NaN
1719,1719,19801719,zum Dorf Gatow,bus,"52.48199565682041, 13.166766484288239",['34E'],NaN


I have two dataframes df_long and df_short, I want to add the information from df_long columns combined_location and combined_identifier to df_short if there is a match between the original_name column from df_long with the stop_name column of df_short. I have to create combined_location and combined_identifier columns in df_short first

In [105]:
for index, row in df_1982.iterrows():
 
    # and row["in_lines"] is in df_stops_total["in_lines
    condition_in_lines = any(all(value in ast.literal_eval(line) for value in ast.literal_eval(row["in_lines"])) for line in df_stops_1980["in_lines"].values)


    if row["stop_name"] in df_stops_1980["stop_name"].values and condition_in_lines:
        # Find matching rows in df_1960
        matching_rows = df_stops_1980[df_stops_1980["stop_name"] == row["stop_name"]]
        
        # Check if those two rows have matching row["type"] values
        matching_rows = matching_rows[matching_rows["type"] == row["type"]]
        
        # Check if there are any matching rows
        if not matching_rows.empty:
            df_1982.at[index, 'location'] = '+ '.join(matching_rows['location'].astype(str).values)
            df_1982.at[index, 'identifier'] = '+ '.join(matching_rows['identifier'].astype(str).values)
            df_1982.at[index, 'stop_id_other'] = '+ '.join(matching_rows['stop_id'].astype(str).values)
            df_1982.at[index, 'previous_in_lines'] = '+ '.join(matching_rows['in_lines'].astype(str).values)
        else:
            # Handle the case where there are no matching rows in df_1960
            df_1982.at[index, 'location'] = ''
            df_1982.at[index, 'identifier'] = ''
            df_1982.at[index, 'stop_description'] = ''
            df_1982.at[index, 'stop_id_other'] = ''
            df_1982.at[index, 'previous_in_lines'] = ''


In [106]:
# Function to split string values containing '+' and convert to a unique string
def split_values(cell_value):
    unique_values = sorted(set(element.strip() for element in cell_value.split('+')))
    return '+'.join(unique_values)

# Apply the function to each cell in the specified columns
df_1982['location'] = df_1982['location'].apply(split_values)
df_1982['identifier'] = df_1982['identifier'].apply(split_values)

In [107]:
df_1982.drop(columns=['line_count', "stop_id_other"], inplace=True)

In [108]:
# Function to concatenate unique values in other columns
def concat_unique_values(series):
    unique_values = set(series)
    return '+ '.join(unique_values)

# Group by specified columns and aggregate the other columns
df_1982 = df_1982.groupby(['stop_name', 'type', 'in_lines', 'location']).agg({
    'identifier': concat_unique_values,
    'previous_in_lines': concat_unique_values,
    'stop_description': concat_unique_values
}).reset_index()

In [109]:
df_1982

,stop_name,type,in_lines,location,identifier,previous_in_lines,stop_description
0,Achtermannstrasse,bus,{'52'},"52.57652468032656, 13.418517187666131",nan,['52'],
1,Adalbertstrasse,bus,"{'78', '78E'}","52.50548699996121, 13.42221404498021",nan,"['78', '75', '78E']",
2,Adamstrasse,bus,"{'97', '13', '35', '34'}",,,,
3,Adamstrasse/Wilhelmstrasse,bus,{'92'},,,,
4,Adenauer Platz,u-bahn,{'7'},,,,
...,...,...,...,...,...,...,...
1775,Zur Zitadelle,bus,"{'99', '13', '55'}","52.53857973555156, 13.212956865853485",nan,['55'],
1776,Zweibrücker Strasse,bus,{'63'},,,,
1777,Zwickauer Damm,bus,{'41'},"52.42412841468055, 13.48547693484881",nan,['41'],
1778,Zwickauer Damm,u-bahn,{'7'},"52.423333333333,13.483888888889",Q572096,['7'],


In [110]:
df_1982.to_csv('1982-stops-combined.csv')